## Import

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 13.0 MB/s eta 0:00:00


In [3]:
pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.5 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import torch
import transformers
#import bitsandbytes as bnb
import os
from transformers import PreTrainedTokenizerFast, AdamW, AutoModelForCausalLM, BitsAndBytesConfig
#from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from tqdm import tqdm

#os.environ["TOKENIZERS_PARALLELISM"] = "true"
#torch.backends.cuda.matmul.allow_tf32=True
#torch.set_float32_matmul_precision('medium')
#torch.backends.cudnn.benchmark = True

## Data Preprocessing

In [5]:
# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/데이콘/한솔데코/train_오타수정.csv')
tokenizer = PreTrainedTokenizerFast.from_pretrained('LDCC/LDCC-SOLAR-10.7B',  eos_token='</s>')#롯데통신이 있었구만 ㅋㅋ;

max_length = 128

formatted_data = []
for _, row in tqdm(data.iterrows()):
  for q_col in ['질문_1', '질문_2']:
    for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
      input_text = row[q_col] + tokenizer.eos_token + row[a_col]
      input_ids = tokenizer.encode(input_text, return_tensors='pt', padding='max_length', truncation=True, max_length=max_length)
      formatted_data.append(input_ids)
print('Done.')


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
644it [00:03, 193.14it/s]

Done.


In [6]:
formatted_data = torch.cat(formatted_data, dim=0)

## Model Fine-tuning

In [ ]:
model_id = "LDCC/LDCC-SOLAR-10.7B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             #torch_dtype=torch.float32,
                                             )

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


config = LoraConfig(#PEFT
    r=8,
    lora_alpha=32,
    #target_modules=["query_key_value"],
    target_modules=[
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=formatted_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
      #  max_steps=50,
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train()

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Step,Training Loss
10,1.847300
20,1.462200
30,1.432600
40,1.389900
50,1.305900
60,1.285300
70,1.319200
80,1.273200
90,1.211500
100,1.276800


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

Step,Training Loss
10,1.847300
20,1.462200
30,1.432600
40,1.389900
50,1.305900
60,1.285300
70,1.319200
80,1.273200
90,1.211500
100,1.276800


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

KeyboardInterrupt: 

# Inference

In [7]:
pip install sentence_transformers

In [8]:
import numpy as np
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [1]:
from transformers import pipeline
checkpoint_path = "/content/drive/MyDrive/데이콘/한솔데코/trained_KoSolarmodel"

# 파이프라인이 답이다.
text_generation = pipeline("text-generation", model=checkpoint_path, return_full_text=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
#from transformers import AutoModelForCausalLM

#checkpoint_path = "/content/drive/MyDrive/데이콘/한솔데코/trained_KoSolarmodel" # 'xxxx'는 실제 체크포인트 번호로 대체하세요.
#llm_model = AutoModelForCausalLM.from_pretrained(checkpoint_path)

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [10]:
llm_model.config.use_cache = False # 캐시 사용 비활성화

In [28]:
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0


cfg_token = 160
cfg_batch = 4096
cfg_ctx = 512
generation_kwargs = {
    "max_length":cfg_token,
    "eos_token_id": tokenizer.eos_token_id,
    "top_k":1 # This is essentially greedy decoding, since the model will always return the highest-probability token. Set this value > 1 for sampling decoding
}

In [10]:
test_data = pd.read_csv('/content/drive/MyDrive/데이콘/한솔데코/test.csv')

1. Pipeline으로 모델 불러옴(return_fulltext=False하기위해) -> input 그대로 넣기
2.

In [33]:
pipe = text_generation("text-classification")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


KeyboardInterrupt: 

In [31]:
res_score=[]
for test_question in test_data['질문']:
  #input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')
  outputs = text_generation(test_question, max_length=cfg_token,eos_token_id=tokenizer.eos_token_id,top_k=1,do_sample=True)  # 샘플링 활성화
  #generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
  #answer_start = generated_text.find(test_question) + len(tokenizer.eos_token)
  #answer_only = generated_text[answer_start:].strip()
  print(outputs)
  answer_only = outputs[0]['generated_text'].replace('\n\nA: ','')
  print(answer_only)

  # 생성된 텍스트를 임베딩
  embedding = model.encode([answer_only])
  res_score.append(embedding)
new_col = ['vec_' + str(i) for i in range(512)]
df_sub = pd.DataFrame(columns=new_col, data=res_score)
df_sub.insert(0, column='id', value=test_data['id'])
df_sub.to_csv('sub.csv', index=False)

[{'generated_text': '\n\nA: 제가  아는  한, 원목사 이딩은 페인트가  아니라 목재 표면을  보호하고  강화하기  위해 목재에 바르는 투 명 코팅제입니다. 이것은  목재에 자연스러운  색조를  유지하면서  햇빛, 비, 기타  환경 요인으로부터  보호합니다. 그러나  시간이  지나면 원목사 이딩은 벗겨지고  재적용이 필요할 수 있습니다. 또한  일부 원목사 이딩 제품 은 냄 새가 나거나  환경적으로  문제가 있을  수 있으므로  사용하기  전에  제품 을 연구하는  것이  좋습니다. 벽 에 페인 트 대신 에 목재 표면을  보호하고  강화하기  위해 목재 스테인을  사용할 수도  있습니다.'}]
제가  아는  한, 원목사 이딩은 페인트가  아니라 목재 표면을  보호하고  강화하기  위해 목재에 바르는 투 명 코팅제입니다. 이것은  목재에 자연스러운  색조를  유지하면서  햇빛, 비, 기타  환경 요인으로부터  보호합니다. 그러나  시간이  지나면 원목사 이딩은 벗겨지고  재적용이 필요할 수 있습니다. 또한  일부 원목사 이딩 제품 은 냄 새가 나거나  환경적으로  문제가 있을  수 있으므로  사용하기  전에  제품 을 연구하는  것이  좋습니다. 벽 에 페인 트 대신 에 목재 표면을  보호하고  강화하기  위해 목재 스테인을  사용할 수도  있습니다.


KeyboardInterrupt: 

In [ ]:
#test_data[0].shape[:2]

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
llm_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(48000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
    